In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
import pandas as pd
import numpy as np
import json

In [2]:
!ls

app.py
data_engineering.ipynb
data_engineering_updated.ipynb
flask_queries.ipynb
GitHub
raw_data
README.md
refugees_and_conflict.sqlite
templates
Untitled.ipynb


In [3]:
engine = create_engine('sqlite:///refugees_and_conflict.sqlite')

In [4]:
Base = automap_base()

In [5]:
Base.prepare(engine, reflect = True)

In [6]:
Base.classes.keys()

['asylum', 'battle_deaths', 'origin']

In [7]:
session = Session(engine)

In [8]:
Asylum = Base.classes.asylum
BattleDeaths = Base.classes.battle_deaths
Origin = Base.classes.origin

In [9]:
samples_df = pd.read_sql_table('battle_deaths', engine)
samples_df

,id,country_name,country_lat,country_lon,country_code,y1990,y1991,y1992,y1993,y1994,...,y2008,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017
0,1,Albania,41.0000,20.0000,ALB,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Algeria,28.0000,3.0000,DZA,0,34,257,800,1916,...,339,505,236,267,256,147,107,110,86,0
2,3,American Samoa,-14.3333,-170.0000,ASM,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Andorra,42.5000,1.6000,AND,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Angola,-12.5000,18.5000,AGO,2558,526,2321,12054,3974,...,0,25,0,0,0,0,0,0,0,0
5,6,Antigua and Barbuda,17.0500,-61.8000,ATG,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Argentina,-34.0000,-64.0000,ARG,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,Armenia,40.0000,45.0000,ARM,0,0,85,37,30,...,0,0,0,0,9,0,0,9,4,0
8,9,Aruba,12.5000,-69.9667,ABW,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Australia,-27.0000,133.0000,AUS,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
country_names = session.query(Asylum.country_name).all()
country_names = [x[0] for x in country_names]
country_names

['Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Polynesia',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Greenland',
 'Grenada',
 'Guam',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hun

In [12]:
df = pd.DataFrame(country_names)
df

,0
0,Albania
1,Algeria
2,American Samoa
3,Andorra
4,Angola
5,Antigua and Barbuda
6,Argentina
7,Armenia
8,Aruba
9,Australia


In [14]:
df.to_csv("countries.csv", encoding='utf-8', index=False)